In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bilstm-q8/pytorch/default/1/bilstm_q8_best.pt
/kaggle/input/bilstm-q3/pytorch/default/1/bilstm_q3_best.pt
/kaggle/input/sep-25-dl-gen-ai-nppe-2/sample_submission.csv
/kaggle/input/sep-25-dl-gen-ai-nppe-2/train.csv
/kaggle/input/sep-25-dl-gen-ai-nppe-2/test.csv


In [2]:
pip install trackio -qq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.9/887.9 kB 14.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.0/23.0 MB 80.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 118.6 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms


In [4]:
# Huggingface Access Adding token to environment variable
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
import os
os.environ['HF_TOKEN']  = user_secrets.get_secret("dra_hf_access_token")

In [5]:
from sklearn.model_selection import train_test_split
from pathlib import Path

train_df = pd.read_csv('/kaggle/input/sep-25-dl-gen-ai-nppe-2/train.csv')
test_df = pd.read_csv('/kaggle/input/sep-25-dl-gen-ai-nppe-2/test.csv')


In [6]:
print(train_df.columns)
print(test_df.columns)

Index(['id', 'seq', 'sst8', 'sst3'], dtype='object')
Index(['id', 'seq'], dtype='object')


In [7]:
train_df.head()

,id,seq,sst8,sst3
0,0,GVGLEGGVQLSPARTRGPEFAAPEQAG,CCCCCCCCSCCCCCCGGGCCCCCCCCC,CCCCCCCCCCCCCCCHHHCCCCCCCCC
1,1,NHGKVKIEHTKWNVEYKVTYNRNVFANHIRSGELASNGYHTTRRTA...,CEEEEEECCTTTEEEEEEEEEEEEEEEEEEEEECSCCSSSCCCCEE...,CEEEEEECCCCCEEEEEEEEEEEEEEEEEEEEECCCCCCCCCCCEE...
2,2,EMRKMLADWKGLSKSDGMLSSEGRTKALWLGEANFSYVPKLDPRAS...,CTHHHHHHHHHSGGGCCCCCCCCCCCCEEECSSCEEEEEEETTCGG...,CCHHHHHHHHHCHHHCCCCCCCCCCCCEEECCCCEEEEEEECCCHH...
3,3,QDNKNGWQIRSDDVWGPDTKDSIQTVEGTRDNVVVYKGPSGYVTAP...,CCCCCCEECTTCCBCTTCTTCCEEEEBCTTTEEEEEETTEEEEEEE...,CCCCCCEECCCCCECCCCCCCCEEEEECCCCEEEEEECCEEEEEEE...
4,4,VPNSRDGGGGNHWNVEFGQLIALIGAAICGVIGGALGGFTAAGSCG...,CEEEEEECCCHHHHHHTHHHHHHHHHHHHHHHHSCCTTCCCCSSCC...,CEEEEEECCCHHHHHHCHHHHHHHHHHHHHHHHCCCCCCCCCCCCC...


In [8]:
test_df.head()

,id,seq
0,0,FFKGSYQKVSNQLLYQANQIQDQTGTITIIRDESGELPEDIKISAG...
1,1,ATTKKNSPFPKVEEAYVSGDANITLFIKRGAHIAQNISSPYVGLDK...
2,2,VRALMLELRSGVREALDALGGVWEITKYLFMVDVPNLESELAFLQR...
3,3,HTWGEAAQEDFTRDIREFRRRISERAAAHPLIYLRNALIADATLRA...
4,4,RFLTVVLDAGLGVKEYHERCMAIYDHVDFHGISNGAEDLWILPGQV...


In [9]:
# Length consistency check
bad_rows = []

for i, row in train_df.iterrows():
    if not (len(row['seq']) == len(row['sst8']) == len(row['sst3'])):
        bad_rows.append(i)

print("Bad rows:", len(bad_rows))

Bad rows: 0


In [10]:
# Amino acid vocabulary
# AMINO_ACIDS = list("ACDEFGHIKLMNPQRSTVWY")
# aa2idx['<UNK>'] = len(aa2idx)  # new index for unknown
# aa2idx = {aa: i + 1 for i, aa in enumerate(AMINO_ACIDS)}  # 0 reserved for padding
# idx2aa = {i: aa for aa, i in aa2idx.items()}


# VOCAB_SIZE = len(aa2idx) + 1  # +1 for padding
# print("Vocab size:", VOCAB_SIZE)


# # Q8 label vocabulary
# Q8_LABELS = sorted(list(set("".join(train_df["sst8"]))))

# q8_2idx = {l: i for i, l in enumerate(Q8_LABELS)}
# idx2q8 = {i: l for l, i in q8_2idx.items()}

# print("Q8 labels:", Q8_LABELS)


# # Q3 label vocabulary
# Q3_LABELS = sorted(list(set("".join(train_df["sst3"]))))

# q3_2idx = {l: i for i, l in enumerate(Q3_LABELS)}
# idx2q3 = {i: l for l, i in q3_2idx.items()}

# print("Q3 labels:", Q3_LABELS)


In [11]:
from torch.utils.data import Dataset

class ProteinDataset(Dataset):
    def __init__(self, df, task="q8"):
        self.seqs = df["seq"].values
        
        if task == "q8":
            self.labels = df["sst8"].values
            self.label2idx = q8_2idx
        else:
            self.labels = df["sst3"].values
            self.label2idx = q3_2idx

    def __len__(self):
        return len(self.seqs)

    def __getitem__(self, idx):
        seq = self.seqs[idx]
        lab = self.labels[idx]

        seq_encoded = torch.tensor(
            [aa2idx.get(a, aa2idx['<UNK>']) for a in seq],  # fix KeyError
            dtype=torch.long
        )

        lab_encoded = torch.tensor(
            [self.label2idx[c] for c in lab],
            dtype=torch.long
        )

        return seq_encoded, lab_encoded


In [12]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm
import pandas as pd
import numpy as np

# TrackIO import
try:
    import trackio
    TRACKIO_AVAILABLE = True
    print("✓ TrackIO available")
except ImportError:
    TRACKIO_AVAILABLE = False
    print("⚠️ TrackIO not available - install with: pip install trackio")

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")


✓ TrackIO available
PyTorch version: 2.6.0+cu124
CUDA available: True


In [13]:
# =============================================================================
# 1️⃣ DATA PREPARATION
# =============================================================================

# Split train/validation
train_data, val_data = train_test_split(train_df, test_size=0.1, random_state=42, shuffle=True)
print(f"\nTrain samples: {len(train_data)} | Val samples: {len(val_data)}")

# Build vocabularies
aa_list = list("ACDEFGHIKLMNPQRSTVWY")
aa2idx = {aa: i+1 for i, aa in enumerate(aa_list)}
aa2idx['<PAD>'] = 0
aa2idx['<UNK>'] = 21

Q8_LABELS = sorted(list(set("".join(train_df["sst8"]))))
q8_2idx = {l: i for i, l in enumerate(Q8_LABELS)}
idx2q8 = {i: l for l, i in q8_2idx.items()}

Q3_LABELS = sorted(list(set("".join(train_df["sst3"]))))
q3_2idx = {l: i for i, l in enumerate(Q3_LABELS)}
idx2q3 = {i: l for l, i in q3_2idx.items()}

print(f"AA vocab size: 22")
print(f"Q8 classes: {len(q8_2idx)} -> {Q8_LABELS}")
print(f"Q3 classes: {len(q3_2idx)} -> {Q3_LABELS}")


Train samples: 6535 | Val samples: 727
AA vocab size: 22
Q8 classes: 8 -> ['B', 'C', 'E', 'G', 'H', 'I', 'S', 'T']
Q3 classes: 3 -> ['C', 'E', 'H']


In [14]:

# =============================================================================
# 2️⃣ DATASET & DATALOADER
# =============================================================================

class ProteinDataset(Dataset):
    def __init__(self, df, task="q8"):
        self.seqs = df["seq"].values
        self.task = task
        if task == "q8":
            self.labels = df["sst8"].values
            self.label2idx = q8_2idx
        else:
            self.labels = df["sst3"].values
            self.label2idx = q3_2idx

    def __len__(self):
        return len(self.seqs)

    def __getitem__(self, idx):
        seq = self.seqs[idx]
        lab = self.labels[idx]
        
        seq_encoded = torch.tensor([aa2idx.get(a, 21) for a in seq], dtype=torch.long)
        lab_encoded = torch.tensor([self.label2idx.get(c, 0) for c in lab], dtype=torch.long)
        
        return seq_encoded, lab_encoded, len(seq_encoded)

def collate_fn(batch):
    seqs, labels, lengths = zip(*batch)
    lengths = torch.tensor(lengths)
    seqs_padded = pad_sequence(seqs, batch_first=True, padding_value=0)
    labels_padded = pad_sequence(labels, batch_first=True, padding_value=-100)
    return seqs_padded, labels_padded, lengths


In [15]:
# =============================================================================
# 3️⃣ MODEL ARCHITECTURE
# =============================================================================

import torch
import torch.nn as nn
import torch.nn.functional as F

class BiLSTM_CNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim,
                 num_layers=2, cnn_filters=[64, 128], kernel_sizes=[3,5], dropout=0.2):
        super().__init__()
        
        # Embedding layer for amino acids
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.dropout_embed = nn.Dropout(dropout)
        
        # -------------------
        # CNN for local patterns
        # -------------------
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=embed_dim, out_channels=f, kernel_size=k, padding=k//2)
            for f, k in zip(cnn_filters, kernel_sizes)
        ])
        
        # -------------------
        # BiLSTM for long-range dependencies
        # -------------------
        lstm_input_dim = sum(cnn_filters)  # CNN output channels concatenated
        self.rnn = nn.LSTM(
            input_size=lstm_input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            bidirectional=True,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        self.layer_norm = nn.LayerNorm(hidden_dim*2)
        self.dropout_out = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        
    def forward(self, x, lengths):
        # -------------------
        # Embedding
        # x: [B, L]
        # -------------------
        embed = self.embedding(x)       # [B, L, embed_dim]
        embed = self.dropout_embed(embed)
        
        # -------------------
        # CNN expects [B, embed_dim, L]
        # -------------------
        cnn_input = embed.transpose(1,2)
        cnn_outs = [F.relu(conv(cnn_input)) for conv in self.convs]  # list of [B, out_channels, L]
        cnn_out = torch.cat(cnn_outs, dim=1).transpose(1,2)          # [B, L, sum(out_channels)]
        
        # -------------------
        # Pack for LSTM
        # -------------------
        packed = nn.utils.rnn.pack_padded_sequence(cnn_out, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_out, _ = self.rnn(packed)
        lstm_out, _ = nn.utils.rnn.pad_packed_sequence(packed_out, batch_first=True)  # [B, L, hidden*2]
        
        # -------------------
        # Layer norm + dropout + classifier
        # -------------------
        lstm_out = self.layer_norm(lstm_out)
        lstm_out = self.dropout_out(lstm_out)
        logits = self.fc(lstm_out)  # [B, L, output_dim]
        
        return logits


In [16]:
# =============================================================================
# 4️⃣ TRAINING FUNCTION
# =============================================================================

def train_model(task="q8", num_epochs=20, batch_size=64, lr=1e-3, use_trackio=True):
    """Train model for either Q8 or Q3 task using BiLSTM+CNN"""
    
    print(f"\n{'='*60}")
    print(f"Training BiLSTM+CNN for {task.upper()} task")
    print(f"{'='*60}")
    
    # Initialize TrackIO
    tracker = None
    if use_trackio and TRACKIO_AVAILABLE:
        try:
            trackio.init(
                project="25-t3-nppe2",
                group=f"bilstm_cnn_{task}",
                name=f"bilstm_cnn_{task}_run"
            )
            trackio.config({
                "model": "BiLSTM_CNN",
                "task": task,
                "vocab_size": 22,
                "embed_dim": 128,
                "hidden_dim": 256,
                "num_layers": 2,
                "cnn_filters": [64,128],
                "kernel_sizes": [3,5],
                "dropout": 0.3,
                "batch_size": batch_size,
                "learning_rate": lr,
                "num_epochs": num_epochs,
                "output_classes": len(q8_2idx) if task == "q8" else len(q3_2idx)
            })
            tracker = True
            print("✓ TrackIO initialized")
        except Exception as e:
            print(f"⚠️ TrackIO initialization failed: {e}")
            tracker = None
    
    # Prepare data
    train_dataset = ProteinDataset(train_data, task=task)
    val_dataset = ProteinDataset(val_data, task=task)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    
    # Model setup
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    output_dim = len(q8_2idx) if task == "q8" else len(q3_2idx)

    model = BiLSTM_CNN(
        vocab_size=22,
        embed_dim=128,
        hidden_dim=256,
        output_dim=output_dim,
        num_layers=2,
        cnn_filters=[64,128],
        kernel_sizes=[3,7,11],
        dropout=0.3
    ).to(device)

    criterion = nn.CrossEntropyLoss(ignore_index=-100)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=2, verbose=True
    )
    
    print(f"Device: {device}")
    print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    # Resume from checkpoint if exists
    checkpoint_path = f"bilstm_cnn_{task}_checkpoint.pt"
    start_epoch = 0
    best_f1 = 0
    
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        start_epoch = checkpoint['epoch']
        best_f1 = checkpoint['best_f1']
        print(f"✓ Resuming from epoch {start_epoch}, best F1: {best_f1:.4f}")
     
    # Training loop
    print("\nStarting training...")
    for epoch in range(start_epoch, num_epochs):
        # Training phase
        model.train()
        total_loss = 0
        
        for seqs, labels, lengths in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            seqs, labels, lengths = seqs.to(device), labels.to(device), lengths.to(device)
            
            optimizer.zero_grad()
            logits = model(seqs, lengths)
            loss = criterion(logits.view(-1, logits.size(-1)), labels.view(-1))
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            total_loss += loss.item()
        
        avg_loss = total_loss / len(train_loader)
        
        # Validation phase
        model.eval()
        all_preds, all_labels = [], []
        
        with torch.no_grad():
            for seqs, labels, lengths in val_loader:
                seqs, labels, lengths = seqs.to(device), labels.to(device), lengths.to(device)
                
                logits = model(seqs, lengths)
                preds = torch.argmax(logits, dim=-1)
                
                mask = labels.view(-1) != -100
                all_preds.extend(preds.view(-1)[mask].cpu().numpy())
                all_labels.extend(labels.view(-1)[mask].cpu().numpy())
        
        val_f1 = f1_score(all_labels, all_preds, average='macro')
        scheduler.step(val_f1)
        
        # Get current learning rate
        current_lr = optimizer.param_groups[0]['lr']
        
        print(f"Epoch {epoch+1} | Loss: {avg_loss:.4f} | Val F1: {val_f1:.4f} | LR: {current_lr:.6f}")
        
        # Log to TrackIO
        if tracker:
            trackio.log({
                "train_loss": avg_loss,
                "val_f1": val_f1,
                "learning_rate": current_lr,
                "epoch": epoch + 1
            })
        
        # Save checkpoint
        if val_f1 > best_f1:
            best_f1 = val_f1
            checkpoint = {
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'best_f1': best_f1,
            }
            torch.save(checkpoint, checkpoint_path)
            torch.save(model.state_dict(), f"bilstm_cnn_{task}_best.pt")
            print(f"  → Best model saved (F1: {best_f1:.4f})")
            
            # Log best metric to TrackIO
            if tracker:
                trackio.log({
                    "best_val_f1": best_f1
                })
    
    # Final summary to TrackIO
    if tracker:
        trackio.log({
            "final_best_f1": best_f1,
            "total_epochs": num_epochs,
            "task": task
        })
        trackio.finish()
        print("✓ TrackIO logging complete")
    
    print(f"\nTraining complete! Best F1: {best_f1:.4f}")
    return model, best_f1


In [17]:
# =============================================================================
# 5️⃣ INFERENCE FUNCTION
# =============================================================================
import os
def predict(model, test_df, task="q8", device="cuda"):
    """Generate predictions for test set"""
    
    class TestDataset(Dataset):
        def __init__(self, df):
            self.seqs = df["seq"].values
        
        def __len__(self):
            return len(self.seqs)
        
        def __getitem__(self, idx):
            seq = self.seqs[idx]
            seq_encoded = torch.tensor([aa2idx.get(a, 21) for a in seq], dtype=torch.long)
            return seq_encoded, len(seq_encoded)
    
    def test_collate_fn(batch):
        seqs, lengths = zip(*batch)
        lengths = torch.tensor(lengths)
        seqs_padded = pad_sequence(seqs, batch_first=True, padding_value=0)
        return seqs_padded, lengths
    
    test_dataset = TestDataset(test_df)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=test_collate_fn)
    
    idx2label = idx2q8 if task == "q8" else idx2q3
    
    model.eval()
    all_predictions = []
    
    with torch.no_grad():
        for seqs, lengths in tqdm(test_loader, desc=f"Predicting {task.upper()}"):
            seqs, lengths = seqs.to(device), lengths.to(device)
            logits = model(seqs, lengths)
            preds = torch.argmax(logits, dim=-1)
            
            # Convert to strings
            for i, length in enumerate(lengths):
                pred_seq = "".join([idx2label[idx.item()] for idx in preds[i][:length]])
                all_predictions.append(pred_seq)
    
    return all_predictions


In [ ]:
# =============================================================================
# 6️⃣ MAIN EXECUTION FOR KAGGLE - TRAIN AND SAVE MODELS
# =============================================================================

import os
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import pandas as pd

# Optional: HF token
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    os.environ["HF_TOKEN"] = user_secrets.get_secret("dra_hf_access_token")
    print("✓ HF token set successfully")
except Exception as e:
    print(f"⚠️ HF token not set: {e}")

# -----------------------------
# TRAINING FUNCTION (SAFE, NO CHECKPOINTS)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:
✓ Closed any existing wandb run
✓ No existing wandb session found

============================================================
Please login with YOUR wandb API key:
============================================================
wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\meekg\_netrc
wandb: Currently logged in as: 23f2002825 (23f2002825-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
Tracking run with wandb version 0.23.1
Run data is saved locally in c:\Users\meekg\Downloads\sep-25-dl-gen-ai-nppe-2\wandb\run-20251216_004517-ai5ufau2
Syncing run bilstm-sst-prediction to Weights & Biases (docs)
View project at https://wandb.ai/23f2002825-iit-madras/protein-secondary-structure
View run at https://wandb.ai/23f2002825-iit-madras/protein-secondary-structure/runs/ai5ufau2
wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.

✓ WandB initialized successfully with YOUR account!
✓ Dashboard URL: https://wandb.ai/23f2002825-iit-madras/protein-secondary-structure/runs/ai5ufau2
✓ Hugging Face transformers loaded!

Loading datasets...
✓ Train dataset: 7262 samples
✓ Test dataset: 1816 samples

C3 classes: ['C', 'E', 'H'] (count: 3)
C8 classes: ['B', 'C', 'E', 'G', 'H', 'I', 'S', 'T'] (count: 8)
Max sequence length: 1632

============================================================
✓ Cell 2 complete - Ready to proceed with training!
============================================================
Using device: cuda
✓ Dataloaders created successfully!
==================================================
Training C3 Model
==================================================
Epoch 1/10 c3 loss: 0.8535
Epoch 2/10 c3 loss: 0.7934
Epoch 3/10 c3 loss: 0.7614
Epoch 4/10 c3 loss: 0.7076
Epoch 5/10 c3 loss: 0.6918
Epoch 6/10 c3 loss: 0.6784
Epoch 7/10 c3 loss: 0.6703
Epoch 8/10 c3 loss: 0.6609

✓ HF token set successfully


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
Q8 Epoch 1: 100%|██████████| 103/103 [00:58<00:00,  1.76it/s]


Epoch 1 | Loss: 1.4473 | Val F1: 0.2742 | LR: 0.001000
  → Best model saved for q8 (F1: 0.2742)


Q8 Epoch 2: 100%|██████████| 103/103 [00:57<00:00,  1.78it/s]


Epoch 2 | Loss: 1.2281 | Val F1: 0.2871 | LR: 0.001000
  → Best model saved for q8 (F1: 0.2871)


...

Epoch 19 | Loss: 0.4662 | Val F1: 0.7018 | LR: 0.000250


Q3 Epoch 20: 100%|██████████| 103/103 [00:58<00:00,  1.77it/s]


Epoch 20 | Loss: 0.4574 | Val F1: 0.7020 | LR: 0.000125

FINAL RESULTS
Q8 F1: 0.3408
Q3 F1: 0.7128
Harmonic Mean: 0.4611


Predicting Q3: 100%|██████████| 29/29 [00:09<00:00,  3.01it/s]


✓ Submission file created: submission.csv
   ID                                               sst8  \
0   0  CCCTTHHHHHHHHHHHHHHHHHHCSEEEEEECCCSTCCCEEEEECC...   
1   1  CCCCCCCCCCEEEEEEECSTTCEEEEEECTTHHHHHHHHCCCTHHH...   
2   2  CTHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCCCSCTHHHHH...   
3   3  CCCTTCHHHCHHHHHHHHHHHHHHHHHHHCCEEHHTHHHHHHHHHH...   
4   4  CEEEEEESCCCTHHHHHHHHHHHHHHHCEEEEECSTTCEEEETSEE...   

                                                sst3  
0  CCCCCCHHHHHHHHHHHHHHHHHCCEEEEEECCCCCCCCEEEEECC...  
1  CCCCCCCCCEEEEEEEECCCCEEEEEEECCCHHHHHHHHCCCCHHH...  
2  CCHHHHHHHHHHHHHHHHHCHHHHHHHHHHHHHHHCCCCHHHHHHH...  
3  CCCCCHHHHHHHHHHHHHHHHHHHHHHHCCCEEECCHHHHHHHHHH...  
4  CEEEEEECCCCCHHHHHHHHHHHHHHHCEEEEECCCCCEEEECCCE...  


In [19]:
# If you want to upload just the model file
# import os
# import shutil

# # Create temp directory for Q8
# os.makedirs("q8_model", exist_ok=True)
# shutil.copy("bilstm_q8_best.pt", "q8_model/bilstm_q8_best.pt")

# kagglehub.model_upload(
#     handle="sreebalajis/bilstm-q8/pytorch/default",
#     local_model_dir="q8_model",
#     version_notes="BiLSTM Q8 model"
# )

# # Same for Q3
# os.makedirs("q3_model", exist_ok=True)
# shutil.copy("bilstm_q3_best.pt", "q3_model/bilstm_q3_best.pt")

# kagglehub.model_upload(
#     handle="sreebalajis/bilstm-q3/pytorch/default",
#     local_model_dir="q3_model",
#     version_notes="BiLSTM Q3 model"
# )

In [20]:
import torch
import torch.nn as nn
import kagglehub
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm

# =============================================================================
# 1️⃣ SETUP - Vocabularies and Model Architecture
# =============================================================================

# Vocabularies (same as training)
aa_list = list("ACDEFGHIKLMNPQRSTVWY")
aa2idx = {aa: i+1 for i, aa in enumerate(aa_list)}
aa2idx['<PAD>'] = 0
aa2idx['<UNK>'] = 21

Q8_LABELS = ['B', 'C', 'E', 'G', 'H', 'I', 'S', 'T']  # From your training
q8_2idx = {l: i for i, l in enumerate(Q8_LABELS)}
idx2q8 = {i: l for l, i in q8_2idx.items()}

Q3_LABELS = ['C', 'E', 'H']  # Update if different
q3_2idx = {l: i for i, l in enumerate(Q3_LABELS)}
idx2q3 = {i: l for l, i in q3_2idx.items()}

# Model Architecture (must match training!)
class BiLSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim, num_layers=2, dropout=0.3):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.rnn = nn.LSTM(
            embed_dim, hidden_dim,
            num_layers=num_layers,
            bidirectional=True,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x, lengths):
        embedded = self.embedding(x)
        embedded = self.dropout(embedded)
        packed = nn.utils.rnn.pack_padded_sequence(
            embedded, lengths.cpu(),
            batch_first=True, enforce_sorted=False
        )
        packed_out, _ = self.rnn(packed)
        out, _ = nn.utils.rnn.pad_packed_sequence(packed_out, batch_first=True)
        out = self.dropout(out)
        logits = self.fc(out)
        return logits

# =============================================================================
# 2️⃣ LOAD MODELS FROM KAGGLEHUB
# =============================================================================

print("Loading models from KaggleHub...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Download Q8 model
q8_path = kagglehub.model_download("sreebalajis/bilstm-q8/pyTorch/default")
model_q8 = BiLSTM(22, 128, 256, len(q8_2idx), num_layers=2, dropout=0.3).to(device)
model_q8.load_state_dict(torch.load(f"{q8_path}/bilstm_q8_best.pt", map_location=device))
model_q8.eval()
print(f"✓ Q8 model loaded from: {q8_path}")

# Download Q3 model
q3_path = kagglehub.model_download("sreebalajis/bilstm-q3/pyTorch/default")
model_q3 = BiLSTM(22, 128, 256, len(q3_2idx), num_layers=2, dropout=0.3).to(device)
model_q3.load_state_dict(torch.load(f"{q3_path}/bilstm_q3_best.pt", map_location=device))
model_q3.eval()
print(f"✓ Q3 model loaded from: {q3_path}")

# =============================================================================
# 3️⃣ PREDICTION FUNCTION
# =============================================================================

def predict(model, test_df, task="q8", device="cuda"):
    """Generate predictions for test set"""
    
    class TestDataset(Dataset):
        def __init__(self, df):
            self.seqs = df["seq"].values
        
        def __len__(self):
            return len(self.seqs)
        
        def __getitem__(self, idx):
            seq = self.seqs[idx]
            seq_encoded = torch.tensor([aa2idx.get(a, 21) for a in seq], dtype=torch.long)
            return seq_encoded, len(seq_encoded)
    
    def test_collate_fn(batch):
        seqs, lengths = zip(*batch)
        lengths = torch.tensor(lengths)
        seqs_padded = pad_sequence(seqs, batch_first=True, padding_value=0)
        return seqs_padded, lengths
    
    test_dataset = TestDataset(test_df)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=test_collate_fn)
    
    idx2label = idx2q8 if task == "q8" else idx2q3
    
    all_predictions = []
    
    with torch.no_grad():
        for seqs, lengths in tqdm(test_loader, desc=f"Predicting {task.upper()}"):
            seqs, lengths = seqs.to(device), lengths.to(device)
            logits = model(seqs, lengths)
            preds = torch.argmax(logits, dim=-1)
            
            for i, length in enumerate(lengths):
                pred_seq = "".join([idx2label[idx.item()] for idx in preds[i][:length]])
                all_predictions.append(pred_seq)
    
    return all_predictions

# =============================================================================
# 4️⃣ GENERATE PREDICTIONS
# =============================================================================

print("\nGenerating predictions...")
q8_preds = predict(model_q8, test_df, task="q8", device=device)
q3_preds = predict(model_q3, test_df, task="q3", device=device)

# =============================================================================
# 5️⃣ CREATE SUBMISSION
# =============================================================================

submission = pd.DataFrame({
    'id': test_df['id'],
    'sst8': q8_preds,
    'sst3': q3_preds
})

submission.to_csv('submission.csv', index=False)
print("\n✓ Submission file created: submission.csv")
print(f"Total test samples: {len(submission)}")
print("\nFirst 5 predictions:")
print(submission.head())

Loading models from KaggleHub...
✓ Q8 model loaded from: /kaggle/input/bilstm-q8/pytorch/default/1
✓ Q3 model loaded from: /kaggle/input/bilstm-q3/pytorch/default/1

Generating predictions...


Predicting Q3: 100%|██████████| 29/29 [00:08<00:00,  3.39it/s]



✓ Submission file created: submission.csv
Total test samples: 1816

First 5 predictions:
   id                                               sst8  \
0   0  CCCCCHHHHHHHHHHHHHHHHHHCSEEEEEECCCTTCCCEEEEECT...   
1   1  CCCCCCCCCCCEEEEEECSTTEEEEEEECTTCEEHHHCCCCEETCC...   
2   2  CCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCSHHHHHHHHH...   
3   3  CCCTHHHHHHHHHHHHHHHHHHHHHHHHCCEEEEHHHHHHHHHHHH...   
4   4  CEEEEEEECCCTHHHHHHHHHHHHHHHCCEEEECSTTEEEEEESEE...   

                                                sst3  
0  CCCCCHHHHHHHHHHHHHHHHHHCCEEEEEECCCCCCCCCEEEECC...  
1  CCCCCCCCCCCEEEEEECCCCEEEEEEECCCHHHHHHHHCCCCCHH...  
2  CCHHHHHHHHHHHHHHHHHCCHHHHHHHHHHHCCCCCCHHHHHHHH...  
3  CCCCCHHHHHHHHHHHHHHHHHHHHHHHHCCEEECCCHHHHHHHHH...  
4  CEEEEEECCCCCHHHHHHHHHHHHHHHCCCCCCCCCCCEEEECCCE...  
